## Created by <a href="https://github.com/yunsuxiaozi">yunsuxiaozi</a> 2024/7/22

#### 比赛链接如下:<a href="https://www.kaggle.com/competitions/jpx-tokyo-stock-exchange-prediction">JPX Tokyo Stock Exchange Prediction</a>.这个方案和我往常解析的方案有点不同,我在看完代码之后在讨论区发现了这个<a href="https://www.kaggle.com/competitions/jpx-tokyo-stock-exchange-prediction/discussion/364227">About the 2nd place solution</a>.这个代码是有错误的,将错误修正之后分数就下降了。也就是说这个代码是靠错误凭运气达到了第二名，第一名据说也是有错误的。但是,一方面我代码已经看完了,不开源就浪费了;另外这也是我第一次接触股票预测的比赛,这个代码对我来说还是有参考意义的,故整理如下。

### 1.导入必要的python库,并固定随机种子。

In [1]:
import pandas as pd#导入csv文件的库
import numpy as np#矩阵运算与科学计算的库
from scipy import stats#统计学分析的python库
import lightgbm as lgb#lightgbm模型
import jpx_tokyo_market_prediction#这个是比赛官方的环境
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

import random#提供了一些用于生成随机数的函数
#设置随机种子,保证模型可以复现
def seed_everything(seed):
    np.random.seed(seed)#numpy的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(seed=2024)

### 2.读取数据集。

In [2]:
#将CSV文件中的"Date"列解析为日期时间格式
train = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv",parse_dates=["Date"])
#RowId就是日期+证券代码,故drop   重复信息
#ExpectedDividend 缺失值占比99%  缺失值
#AdjustmentFactor=1的占比:0.9996   一列几乎只有唯一值
#'SupervisionFlag'为False占比:0.9997 一列几乎只有唯一值
#dropna,去掉有缺失值的行(不确定线性插值会不会更好)
train=train.drop(columns=['RowId','ExpectedDividend','AdjustmentFactor','SupervisionFlag']).dropna().reset_index(drop=True)
print(f"len(train):{len(train)}")
#对测试数据进行同样的操作,测试数据要全部预测,所以不能dropna.
test = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/secondary_stock_prices.csv",parse_dates=["Date"])
test=test.drop(columns=['RowId','ExpectedDividend','AdjustmentFactor','SupervisionFlag'])

train.head()

len(train):2324923


,Date,SecuritiesCode,Open,High,Low,Close,Volume,Target
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,0.000730
1,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,0.012324
2,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,0.006154
3,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,0.011053
4,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,0.003026


### 3.特征工程。这里的特征工程存在错误。它应该groupby 'SecuritiesCode'来考虑移动特征,而它这里只是20行移动。

In [3]:
def add_features(feats):
    #股票相比1个月,2个月,3个月的回报率 pct_change (v_t-v_{t-1}}/v_{t-1})
    feats["return_1month"] = feats["Close"].pct_change(20)
    feats["return_2month"] = feats["Close"].pct_change(40)
    feats["return_3month"] = feats["Close"].pct_change(60)
    
    #股票的波动性,取log可能和长尾分布有关,diff就是作差,rolling是考虑一段时间,std是方差
    feats["volatility_1month"] = (
        np.log(feats["Close"]).diff().rolling(20).std()
    )
    feats["volatility_2month"] = (
        np.log(feats["Close"]).diff().rolling(40).std()
    )
    feats["volatility_3month"] = (
        np.log(feats["Close"]).diff().rolling(60).std()
    )
    
    #股票的收盘价/股票一个月收盘价的移动平均
    feats["MA_gap_1month"] = feats["Close"] / (
        feats["Close"].rolling(20).mean()
    )
    feats["MA_gap_2month"] = feats["Close"] / (
        feats["Close"].rolling(40).mean()
    )
    feats["MA_gap_3month"] = feats["Close"] / (
        feats["Close"].rolling(60).mean()
    )
    
    return feats

#将缺失值,np.inf,-np.inf都转成0
def fill_nan_inf(df):
    df = df.fillna(0)
    df = df.replace([np.inf, -np.inf], 0)
    return df
train = add_features(train)
train=fill_nan_inf(train)
test = add_features(test)
test=fill_nan_inf(test)
train.head()

,Date,SecuritiesCode,Open,High,Low,Close,Volume,Target,return_1month,return_2month,return_3month,volatility_1month,volatility_2month,volatility_3month,MA_gap_1month,MA_gap_2month,MA_gap_3month
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,0.000730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,0.012324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,0.006154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,0.011053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,0.003026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 4.构造训练验证数据集.这里分别选择了1000支股票。

In [4]:
#Target是标签,date是字符串,SecuritiesCode没什么用
features =list(train.drop(['Target','Date','SecuritiesCode'],axis=1).columns)  

#每个证券代码的max_target-min_target排序
group=(train.groupby('SecuritiesCode')['Target'].max()-train.groupby('SecuritiesCode')['Target'].min()).sort_values()
#取max_target-min_target最小的1000个股票
list_spred_h=list(group[:1000].index)
#剩下的股票
list_spred_l=list(group[1000:].index)
print(f"len(train_securitiescode):{len(list_spred_h)},len(valid_securitiescode):{len(list_spred_l)}")

train_X=train[train['SecuritiesCode'].isin(list_spred_h)][features]
train_y=train[train['SecuritiesCode'].isin(list_spred_h)]["Target"]
valid_X=train[train['SecuritiesCode'].isin(list_spred_l)][features]
valid_y=train[train['SecuritiesCode'].isin(list_spred_l)]["Target"]

tr_dataset = lgb.Dataset(train_X,train_y,feature_name = features)
vl_dataset = lgb.Dataset(valid_X,valid_y,feature_name = features)

len(train_securitiescode):1000,len(valid_securitiescode):1000


### 5.模型的训练。

In [5]:
def feval_pearsonr(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'pearsonr', stats.pearsonr(y_true, y_pred)[0], True

#lgb模型的参数
params_lgb = {'learning_rate': 0.005,
              'metric':'None',
              'objective': 'regression',
              'boosting': 'gbdt',
              'verbosity': 0,
              'n_jobs': -1,
              'force_col_wise':True}

model = lgb.train(params = params_lgb, 
                train_set = tr_dataset, 
                valid_sets = [tr_dataset, vl_dataset], 
                num_boost_round = 1000, 
                feval=feval_pearsonr,
                callbacks=[ lgb.early_stopping(stopping_rounds=300, verbose=True), 
                           lgb.log_evaluation(period=100)])  

Training until validation scores don't improve for 300 rounds
[100]	training's pearsonr: 0.0564282	valid_1's pearsonr: 0.0108009
[200]	training's pearsonr: 0.0680563	valid_1's pearsonr: 0.0134107
[300]	training's pearsonr: 0.0761529	valid_1's pearsonr: 0.0142165
[400]	training's pearsonr: 0.082453	valid_1's pearsonr: 0.0146069
[500]	training's pearsonr: 0.0883774	valid_1's pearsonr: 0.0147148
[600]	training's pearsonr: 0.0938508	valid_1's pearsonr: 0.0148582
[700]	training's pearsonr: 0.0986576	valid_1's pearsonr: 0.0148196
[800]	training's pearsonr: 0.103034	valid_1's pearsonr: 0.0146873
[900]	training's pearsonr: 0.106989	valid_1's pearsonr: 0.0146611
Early stopping, best iteration is:
[606]	training's pearsonr: 0.0942029	valid_1's pearsonr: 0.0148682


### 6.模型的推理.这里需要将数值转换成rank.

In [6]:
sample_submission = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/sample_submission.csv")
#创建比赛的环境
env = jpx_tokyo_market_prediction.make_env()
#遍历测试数据的一个迭代器
iter_test = env.iter_test()

def add_rank(df):
    df["Rank"] = df.groupby("Date")["Target"].rank(ascending=False, method="first") - 1 
    df["Rank"] = df["Rank"].astype("int")
    return df


for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:  
    prices = add_features(prices)
    prices['Target'] = model.predict(fill_nan_inf(prices)[features])
    prices['target_median']=prices.groupby("Date")["Target"].transform('median')
    prices.loc[prices['SecuritiesCode'].isin(list_spred_h),'Target']=prices['target_median']
    prices = add_rank(prices)
    sample_prediction['Rank'] = prices['Rank']
    env.predict(sample_prediction)
    
sample_prediction.head()

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


,Date,SecuritiesCode,Rank
0,2021-12-07,1301,497
1,2021-12-07,1332,498
2,2021-12-07,1333,499
3,2021-12-07,1375,500
4,2021-12-07,1376,1916
